<a href="https://colab.research.google.com/github/dancodeur/LunarLander_Q-learning/blob/main/Rosalie_Ornella_Dan_TD_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install swig
!pip install gymnasium[box2d]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 49 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 1s (1,679 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 123635 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubu

In [ ]:
import gym
import numpy as np
import gym
import numpy as np
from gym.wrappers.monitoring.video_recorder import VideoRecorder
import PIL
import matplotlib.pyplot as plt
from IPython import display
import time

In [ ]:
# Initialisation de l'environnement
env = gym.make("LunarLander-v2")

/usr/local/lib/python3.10/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-pa

In [ ]:
# Définition des limites de l'état
state_bounds = np.array([
    [-1.5, 1.5],     # Position x
    [-0.5, 1.5],     # Position y
    [-2.0, 2.0],     # Vitesse x
    [-2.0, 2.0],     # Vitesse y
    [-np.pi, np.pi], # Angle
    [-5.0, 5.0],     # Vitesse angulaire
    [0, 1],          # Contact jambe gauche
    [0, 1]           # Contact jambe droite
])

# Implémentation du codeur de tuile
class FeatureTransformer:
    def __init__(self, num_tilings, num_tiles, state_bounds, max_size=4096):
        self.num_tilings = num_tilings
        self.num_tiles = num_tiles
        self.state_bounds = state_bounds
        self.state_dim = len(state_bounds)
        self.tile_width = (state_bounds[:,1] - state_bounds[:,0]) / (num_tiles - 1)
        self.offsets = [np.linspace(0, self.tile_width[i], num_tilings) for i in range(self.state_dim)]
        self.max_size = int(max_size)  # Nombre maximum de caractéristiques

    def get_feature_indices(self, state):
        feature_indices = []
        for tiling in range(self.num_tilings):
            indices = []
            for i in range(self.state_dim):
                s = state[i]
                low = self.state_bounds[i, 0]
                high = self.state_bounds[i, 1]
                tile_width = self.tile_width[i]
                offset = self.offsets[i][tiling]
                s_offset = s + offset
                # Gérer les valeurs hors limites
                if s_offset < low:
                    s_offset = low
                elif s_offset > high:
                    s_offset = high - 1e-4
                index = int((s_offset - low) / tile_width)
                indices.append(index)
            tile_index = (tiling, *indices)
            # Utiliser le hachage pour obtenir un indice de caractéristique unique
            feature_index = hash(tile_index) % self.max_size
            feature_indices.append(feature_index)
        return feature_indices


In [ ]:
# Initialisation des hyperparamètres et des poids
num_tilings = 8
num_tiles = 8
max_size = 4096

feature_transformer = FeatureTransformer(num_tilings, num_tiles, state_bounds, max_size)

alpha = 0.1 / num_tilings  # Taux d'apprentissage
gamma = 0.99               # Facteur de discount
epsilon = 1.0              # Taux d'exploration initial

num_episodes = 1000

weights = np.zeros((feature_transformer.max_size, env.action_space.n))

# Fonction pour obtenir les valeurs Q
def get_Q_values(state, weights, feature_transformer):
    feature_indices = feature_transformer.get_feature_indices(state)
    Q_values = np.zeros(env.action_space.n)
    for a in range(env.action_space.n):
        Q_values[a] = np.sum(weights[feature_indices, a])
    return Q_values

In [ ]:
# Boucle d'entraînement du Q-learning
for episode in range(num_episodes):
    state = env.reset()
    done = False
    total_reward = 0
    while not done:
        # Obtenir les indices des caractéristiques
        feature_indices = feature_transformer.get_feature_indices(state)

        # Sélection de l'action avec la politique epsilon-greedy
        if np.random.rand() < epsilon:
            action = env.action_space.sample()
        else:
            Q_values = np.array([np.sum(weights[feature_indices, a]) for a in range(env.action_space.n)])
            action = np.argmax(Q_values)

        # Effectuer l'action
        next_state, reward, done, info = env.step(action)
        total_reward += reward

        # Obtenir les indices des caractéristiques du prochain état
        next_feature_indices = feature_transformer.get_feature_indices(next_state)

        # Calcul du TD target
        if done:
            td_target = reward
        else:
            next_Q_values = np.array([np.sum(weights[next_feature_indices, a]) for a in range(env.action_space.n)])
            td_target = reward + gamma * np.max(next_Q_values)

        # Calcul du TD error
        current_Q = np.sum(weights[feature_indices, action])
        td_error = td_target - current_Q

        # Mise à jour des poids
        for idx in feature_indices:
            weights[idx, action] += alpha * td_error

        # Passage à l'état suivant
        state = next_state

    # Diminuer epsilon pour réduire l'exploration au fil du temps
    epsilon = max(epsilon * 0.995, 0.01)

    print(f"Épisode {episode+1}, Récompense totale: {total_reward}")

# Sauvegarder les poids
np.save('weights.npy', weights)

Épisode 1, Récompense totale: -319.65269947264983
Épisode 2, Récompense totale: -203.87411498985273
Épisode 3, Récompense totale: -203.65060053004842
Épisode 4, Récompense totale: -223.18309009660396
Épisode 5, Récompense totale: -114.4854471435412
Épisode 6, Récompense totale: -119.66450553148711
Épisode 7, Récompense totale: -98.18887014509299
Épisode 8, Récompense totale: -151.77318220464946
Épisode 9, Récompense totale: -126.18496948545072
Épisode 10, Récompense totale: -143.1972220073789
Épisode 11, Récompense totale: -185.27354665764923
Épisode 12, Récompense totale: -154.3574490180802
Épisode 13, Récompense totale: -203.8969301632375
Épisode 14, Récompense totale: -73.01693659732317
Épisode 15, Récompense totale: -56.391380000450376
Épisode 16, Récompense totale: -208.0017521495414
Épisode 17, Récompense totale: -376.2415637168596
Épisode 18, Récompense totale: -250.99186078602327
Épisode 19, Récompense totale: -64.50084976829338
Épisode 20, Récompense totale: -265.4661715053401

In [ ]:
# Charger les poids (si nécessaire)
weights = np.load('weights.npy')

# Réinitialiser epsilon pour désactiver l'exploration pendant le test
epsilon = 0

env = gym.make("LunarLander-v2", render_mode='rgb_array')
video = VideoRecorder(env, "trained_agent.mp4")

num_test_episodes = 1
for episode in range(num_test_episodes):
    state = env.reset()
    done = False
    total_reward = 0
    while not done:
        feature_indices = feature_transformer.get_feature_indices(state)
        Q_values = np.array([np.sum(weights[feature_indices, a]) for a in range(env.action_space.n)])
        action = np.argmax(Q_values)
        state, reward, done, info = env.step(action)
        total_reward += reward
        # Enregistrer la frame
        video.capture_frame()
    print(f"Épisode de test {episode+1}, Récompense totale: {total_reward}")

video.close()
env.close()

/usr/local/lib/python3.10/dist-packages/gym/wrappers/monitoring/video_recorder.py:101: DeprecationWarning: WARN: <class 'gym.wrappers.monitoring.video_recorder.VideoRecorder'> is marked as deprecated and will be removed in the future.
  logger.deprecation(


Épisode de test 1, Récompense totale: -105.93648191863458


In [ ]:
# Afficher la vidéo
from IPython.display import HTML
from base64 import b64encode

mp4 = open('trained_agent.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=600 controls>
    <source src="%s" type="video/mp4">
</video>
""" % data_url)